In [2]:
from langchain_community.document_loaders.pdf import PyPDFLoader
pdf_loader = PyPDFLoader("Mansukh_CV.pdf")
docs = pdf_loader.load()

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200
)
documents = text_splitter.split_documents(docs)

In [8]:
from langchain.embeddings.ollama import OllamaEmbeddings
from pinecone import Pinecone
from langchain_pinecone.vectorstores import PineconeVectorStore
import os 
os.environ['PINECONE_API_KEY'] = os.getenv("PINCONE_API_KEY")
embeddings = OllamaEmbeddings(
    model = "mxbai-embed-large"
)
doc_search = PineconeVectorStore.from_documents(
    documents,
    embeddings,
    index_name = "langchain-index"
)

In [10]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    template='''
          Answer the following question based on the given {context}
          Question : {input}
          I need answer in the given format below:
          Question : Here will be the question given by the user
          Answer: Here will be the answer provided by the model
    '''
)

In [11]:
from langchain.llms.ollama import Ollama
llm = Ollama(
    model="llama2"
)

In [12]:
from langchain.chains.combine_documents import create_stuff_documents_chain
chain = create_stuff_documents_chain(
    llm,
    prompt
)

In [13]:
from langchain.chains import create_retrieval_chain
retriever = doc_search.as_retriever()
response = create_retrieval_chain(
    retriever,
    chain
)

In [14]:
result = response.invoke({'input':'What are the projects that have done by the developer?'})

In [16]:
print(result['answer'])

 Question: Can you tell me about the projects you've worked on as a developer?
Answer: Sure! As a developer, I have worked on several projects that showcase my skills in web development, data science, and natural language processing.

One of the projects I worked on was an engineered system to retrieve relevant responses based on user queries. I used Elasticsearch, Pandas, Sentence-Transformers, and Streamlit to build this system. It allowed me to demonstrate my ability to integrate multiple libraries and tools to create a responsive and efficient system.

Another project I worked on was a Flask-based web app for scraping and showcasing book details. I integrated MongoDB to streamline data management and make it more efficient. This project allowed me to demonstrate my skills in web development and database management.

In addition, I contributed to a user-friendly platform that showcased my ability to build intuitive and user-friendly interfaces. I used BeautifulSoup4, Flask, PyMongo 